In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ast

# Data

In [4]:
def clean_data(df: pd.DataFrame) -> pd.DataFrame:
    '''
    Clean data by:
    - Removing or imputing NaN
    '''
    ## cleaning data from ball_by_ball dataframe
    # drop duplicates
    df = df.drop_duplicates()
    # assign the good value to overs
    df["overs"] = df["overs"] + 1
    # cleaning the string values
    # batter
    df["batter"] = [text.strip() for text in df["batter"]] # removing space
    df["batter"] = [text.lower() for text in df["batter"]] # lowercase
    # bowler
    df["bowler"] = [text.strip() for text in df["bowler"]]
    df["bowler"] = [text.lower() for text in df["bowler"]]
    # non-striker
    df.rename(columns={"non-striker": "non_striker"}, inplace=True) #renaming the non_striker
    df["non_striker"] = [text.strip() for text in df["non_striker"]]
    df["non_striker"] = [text.lower() for text in df["non_striker"]]
    # extra-types
    df["extra_type"] = df["extra_type"].fillna("noextra") # filling the NaN values
    # player out
    df["player_out"] = df["player_out"].fillna("noplayerout") # filling the NaN values
    df["player_out"] = [text.strip() for text in df["player_out"]]
    df["player_out"] = [text.lower() for text in df["player_out"]]
    # kind
    df["kind"] = df["kind"].fillna("nokind") # filling the NaN values
    df["kind"] = [text.strip() for text in df["kind"]]
    df["kind"] = [text.lower() for text in df["kind"]]
    # fielders involved
    df["fielders_involved"] = df["fielders_involved"].fillna("nofieldersinvolved") ## filling the NaN values
    df["fielders_involved"] = [text.strip() for text in df["fielders_involved"]]
    df["fielders_involved"] = [text.lower() for text in df["fielders_involved"]]
    # batting team
    df["BattingTeam"] = [text.strip() for text in df["BattingTeam"]]
    df["BattingTeam"] = [text.lower() for text in df["BattingTeam"]]
    # some teams change their name since 2008
    team_name_dict = {'rising pune supergiants': 'rising pune supergiant',
                  'kings xi punjab': 'punjab kings',
                  'delhi daredevils': 'delhi capitals'}
    df['BattingTeam'] = df['BattingTeam'].replace(team_name_dict)

    ## cleaning data from match dataframe
    # Adding the correct City for the matching Venue
    df.loc[df.Venue == 'Dubai International Cricket Stadium','City'] = 'Dubai'
    df.loc[df.Venue == 'Sharjah Cricket Stadium','City'] = 'Sharjah'
    # Creating a dictionary of duplicated Venues so only one entry is created for each venue
    venues_dict = {'Arun Jaitley Stadium, Delhi': 'Arun Jaitley Stadium',
                'Brabourne Stadium, Mumbai': 'Brabourne Stadium',
                'Dr DY Patil Sports Academy, Mumbai': 'Dr DY Patil Sports Academy',
                'Eden Gardens, Kolkata': 'Eden Gardens',
                'M Chinnaswamy Stadium': 'M.Chinnaswamy Stadium',
                'MA Chidambaram Stadium, Chepauk': 'MA Chidambaram Stadium',
                'MA Chidambaram Stadium, Chepauk, Chennai': 'MA Chidambaram Stadium',
                'Maharashtra Cricket Association Stadium, Pune': 'Maharashtra Cricket Association Stadium',
                'Punjab Cricket Association IS Bindra Stadium, Mohali': 'Punjab Cricket Association IS Bindra Stadium',
                'Punjab Cricket Association Stadium, Mohali': 'Punjab Cricket Association IS Bindra Stadium',
                'Rajiv Gandhi International Stadium, Uppal': 'Rajiv Gandhi International Stadium',
                'Wankhede Stadium, Mumbai': 'Wankhede Stadium',
                'Zayed Cricket Stadium, Abu Dhabi': 'Sheikh Zayed Stadium'}
    df['Venue'] = df['Venue'].replace(venues_dict)
    # Two teams have changed their name over the period the dataset covers
    team_name_changes = {'Kings XI Punjab': 'Punjab Kings',
                        'Delhi Daredevils': 'Delhi Capitals'}
    df['Team1'] = df['Team1'].replace(team_name_changes)
    # Updating the team name for Rising Pune Supergiant
    team_name_dict = {'Rising Pune Supergiants': 'Rising Pune Supergiant'}
    df['Team1'] = df['Team1'].replace(team_name_dict)
    # Updating the other team name columns
    df['Team2'] = df['Team2'].replace(team_name_changes)
    df['TossWinner'] = df['TossWinner'].replace(team_name_changes)
    df['WinningTeam'] = df['WinningTeam'].replace(team_name_changes)
    df['Team2'] = df['Team2'].replace(team_name_dict)
    df['TossWinner'] = df['TossWinner'].replace(team_name_dict)
    df['WinningTeam'] = df['WinningTeam'].replace(team_name_dict)
    # Identifying and removing 4 "NoResult" Values
    df['WinningTeam'] = df['WinningTeam'].fillna('NoResults')
    df['Player_of_Match'] = df['Player_of_Match'].fillna('NoResults')
    df['SuperOver'] = df['SuperOver'].fillna('NoResults')
    df.loc[df['WinningTeam'].isna(), 'method'] = 'NoResults'
    df.loc[df['WinningTeam'].isna(), 'Margin'] = 0
    # The method column held no useful information and was dropped
    df = df.drop(columns='method')
    # Results from margin related to Superover and have been filled with 0
    df['Margin'] = df['Margin'].fillna(0)

    # Updating dates so we are able to identify seasonal changes in data
    season_dict = {'2020/21': '2020',
                '2009/10': '2010',
                '2007/08': '2008'}

    df['Season'] = df['Season'].replace(season_dict)
    # Selecting columns to strip
    df_columns = list(df.select_dtypes(include='object').columns)
    # Stripping the data for each column
    for column in df_columns:
        df[column] = [text.strip().lower() for text in df[column]]

    df['Team1Players'] = df['Team1Players'].apply(lambda x: ast.literal_eval(x))
    df['Team2Players'] = df['Team2Players'].apply(lambda x: ast.literal_eval(x))

    print("✅ data cleaned")
    return df


def feature_engineer_old(df: pd.DataFrame) -> pd.DataFrame:
    '''
    This is where we create the new features for the model to be combined with the engineered
    player features.
    They should be:
    - team_1_win --> 1 or 0 if team 1 one the match
    - innings_total --> to be used to calculate batting avgs etc.
    - team_1_batting_average --> the average amount of runs that team1 scores in a match
    - team_2_batting_average --> the average amount of runs that team2 scores in a match
    - team_1_toss_winner --> 1 or 0 if team1 won the toss
    - match_importance --> ordinal value given to the importance of the match. Final or regular etc.
    - team_1_points_against_avg --> the average number of point opposition teams score agains team_1
    - team_2_points_against_avg --> the average number of point opposition teams score agains team_2
    - team_1_avg_mvp --> average number of times that team_1 has MVP
    - team_2_avg_mvp --> average number of times that team_2 has MVP
    '''
    # create the binary column Team1_Win
    df['Team1_Win'] = (df['Team1'] == df['WinningTeam']).astype(int)
    # create the innings total
    df['innings_total'] = df.groupby(['ID', 'innings'])['total_run'].transform('sum')
    df = df[(df['innings'] == 1) | (df['innings'] == 2)]
    # Convert the innings totals into columns

    # Group by 'ID' and 'innings' and calculate innings_total
    innings_totals = df.groupby(['ID', 'innings'])['innings_total'].mean().unstack()
    innings_totals.columns = ['Team1_innings_total', 'Team2_innings_total']

    # Merge the innings_totals DataFrame back into new_df on 'ID'
    df = pd.merge(df, innings_totals, left_on='ID', right_index=True)
    df['team_batting_average'] = df.groupby('BattingTeam')['total_run'].transform('sum') / df.groupby('BattingTeam')['ID'].transform('nunique')
    # Extract the relevant columns
    selected_columns = ['ID', 'innings', 'Team1', 'Team2', 'team_batting_average', 'MatchNumber','WinningTeam', 'innings_total', 'City', 'Date', 'Venue', 'TossWinner', 'TossDecision', 'Team1Players', 'Team2Players']
    new_df = df[selected_columns].copy()
    new_df.head()

    # Filter innings 1 (Team1's innings)
    team_a_data = new_df[new_df['innings'] == 1][['ID','Team1', 'team_batting_average' ]]
    team_a_data.rename(columns={'team_batting_average': 'Team_1_batting_average'}, inplace=True)

    # Filter innings 2 (Team1's innings)
    team_b_data = new_df[new_df['innings'] == 2][['ID', 'Team2', 'team_batting_average']]
    team_b_data.rename(columns={'team_batting_average': 'Team_2_batting_average'}, inplace=True)

    # Merge the data from innings 1 and innings 2 for each match ID
    merged_data = pd.merge(team_a_data, team_b_data, on='ID')
    # Drop duplicates based on 'ID' column and keep the first occurrence
    final_data = merged_data.drop_duplicates(subset='ID', keep='first')
    # Reset the index
    final_data.reset_index(drop=True, inplace=True)
    # drop the duplicate columns
    final_data.drop(columns=["Team1", "Team2"], inplace=True)
    # merge with the main df
    df = pd.merge(df, final_data, on = "ID")
    # drop the useless column
    df.drop(columns=['team_batting_average'], inplace=True)
    # new binary column for the toss winner
    df['team_1_toss_winner'] = (df['Team1'] == df['TossWinner']).astype(int)

    # Apply the function
    df['MatchImportance'] = df['MatchNumber'].apply(map_match_number)
    # Calculate the average points scored against a each team
    df['Team1_points_against_avg'] = df.groupby('Team1')['Team2_innings_total'].transform('mean')
    df['Team2_points_against_avg'] = df.groupby('Team2')['Team1_innings_total'].transform('mean')
    # Calculate the average number of times that team_1 has MVP

    df['Team_MVP'] = df.apply(lambda row: get_match_winner(row['Player_of_Match'], row['Team1Players'], row['Team2Players']), axis=1)

    df['Team_MVP'] = df.apply(replace_team_mvp_with_name, axis=1)
    team_mvp_counts = df['Team_MVP'].value_counts().reset_index()

    team_mvp_counts.columns = ['Team_MVP', 'MVP_Count']

    # Create the columns Team1_MVP_appearances e Team2_MVP_appearances
    df = df.merge(team_mvp_counts, left_on='Team1', right_on='Team_MVP', how='left').fillna(0)
    df.rename(columns={'MVP_Count': 'Team1_MVP_appearances'}, inplace=True)

    df = df.merge(team_mvp_counts, left_on='Team2', right_on='Team_MVP', how='left').fillna(0)
    df.rename(columns={'MVP_Count': 'Team2_MVP_appearances'}, inplace=True)

    # Drop columns
    df.drop(['Team_MVP_x', 'Team_MVP_y', 'Team_MVP'], axis=1, inplace=True)

    # Now, let's create the average of MVP appearences
    total_games_team1 = df['Team1'].value_counts()
    total_games_team2 = df['Team2'].value_counts()

    total_games = total_games_team2 + total_games_team1

    df['Team1_MVP_average'] = df['Team1_MVP_appearances'] / total_games[df['Team1']].values
    df['Team2_MVP_average'] = df['Team2_MVP_appearances'] / total_games[df['Team2']].values
    df = df.drop(columns = ['Team1_MVP_appearances', 'Team2_MVP_appearances'])

    # Drop duplicates based on 'ID' column and keep the first occurrence
    final_data = df.drop_duplicates(subset='ID', keep='first')

    print(f"✅ New featured engineered new shape is {final_data.shape}")

    return final_data

# define the importance of each match
def map_match_number(value):
            if isinstance(value, int) or value.isnumeric():
                return 1
            elif value in ['qualifier 2', 'eliminator', 'qualifier 1', 'qualifier', 'elimination final', '3rd place play-off', 'semi final']:
                return 2
            elif value == 'final':
                return 3
            else:
                return 0

def get_match_winner(player_of_match, team1_players, team2_players):
            if pd.isna(player_of_match):
                return 'N/A'

            if pd.isna(team1_players):
                team1_players = []

            if pd.isna(team2_players):
                team2_players = []

            if player_of_match in team1_players:
                return 'Team1'
            elif player_of_match in team2_players:
                return 'Team2'
            else:
                return 'N/A'

def replace_team_mvp_with_name(row):
            if row['Team_MVP'] == 'Team1':
                return row['Team1']
            elif row['Team_MVP'] == 'Team2':
                return row['Team2']
            else:
                return row['Team_MVP']






"""import numpy as np
import pandas as pd
import ast
#from google.cloud import bigquery
from pathlib import Path

'''
This module is used for cleaning data that can be
be used to pass into the model in the model.py function
'''

def clean_data(df: pd.DataFrame) -> pd.DataFrame:
    '''
    Clean data by:
    - Removing or imputing NaN
    '''
    ## cleaning data from ball_by_ball dataframe
    # drop duplicates
    df = df.drop_duplicates()
    # assign the good value to overs
    df["overs"] = df["overs"] + 1
    # cleaning the string values
    # batter
    df["batter"] = [text.strip() for text in df["batter"]] # removing space
    df["batter"] = [text.lower() for text in df["batter"]] # lowercase
    # bowler
    df["bowler"] = [text.strip() for text in df["bowler"]]
    df["bowler"] = [text.lower() for text in df["bowler"]]
    # non-striker
    df.rename(columns={"non-striker": "non_striker"}, inplace=True) #renaming the non_striker
    df["non_striker"] = [text.strip() for text in df["non_striker"]]
    df["non_striker"] = [text.lower() for text in df["non_striker"]]
    # extra-types
    df["extra_type"] = df["extra_type"].fillna("noextra") # filling the NaN values
    # player out
    df["player_out"] = df["player_out"].fillna("noplayerout") # filling the NaN values
    df["player_out"] = [text.strip() for text in df["player_out"]]
    df["player_out"] = [text.lower() for text in df["player_out"]]
    # kind
    df["kind"] = df["kind"].fillna("nokind") # filling the NaN values
    df["kind"] = [text.strip() for text in df["kind"]]
    df["kind"] = [text.lower() for text in df["kind"]]
    # fielders involved
    df["fielders_involved"] = df["fielders_involved"].fillna("nofieldersinvolved") ## filling the NaN values
    df["fielders_involved"] = [text.strip() for text in df["fielders_involved"]]
    df["fielders_involved"] = [text.lower() for text in df["fielders_involved"]]
    # batting team
    df["BattingTeam"] = [text.strip() for text in df["BattingTeam"]]
    df["BattingTeam"] = [text.lower() for text in df["BattingTeam"]]
    # some teams change their name since 2008
    team_name_dict = {'rising pune supergiants': 'rising pune supergiant',
                  'kings xi punjab': 'punjab kings',
                  'delhi daredevils': 'delhi capitals'}
    df['BattingTeam'] = df['BattingTeam'].replace(team_name_dict)

    ## cleaning data from match dataframe
    # Adding the correct City for the matching Venue
    df.loc[df.Venue == 'Dubai International Cricket Stadium','City'] = 'Dubai'
    df.loc[df.Venue == 'Sharjah Cricket Stadium','City'] = 'Sharjah'
    # Creating a dictionary of duplicated Venues so only one entry is created for each venue
    venues_dict = {'Arun Jaitley Stadium, Delhi': 'Arun Jaitley Stadium',
                'Brabourne Stadium, Mumbai': 'Brabourne Stadium',
                'Dr DY Patil Sports Academy, Mumbai': 'Dr DY Patil Sports Academy',
                'Eden Gardens, Kolkata': 'Eden Gardens',
                'M Chinnaswamy Stadium': 'M.Chinnaswamy Stadium',
                'MA Chidambaram Stadium, Chepauk': 'MA Chidambaram Stadium',
                'MA Chidambaram Stadium, Chepauk, Chennai': 'MA Chidambaram Stadium',
                'Maharashtra Cricket Association Stadium, Pune': 'Maharashtra Cricket Association Stadium',
                'Punjab Cricket Association IS Bindra Stadium, Mohali': 'Punjab Cricket Association IS Bindra Stadium',
                'Punjab Cricket Association Stadium, Mohali': 'Punjab Cricket Association IS Bindra Stadium',
                'Rajiv Gandhi International Stadium, Uppal': 'Rajiv Gandhi International Stadium',
                'Wankhede Stadium, Mumbai': 'Wankhede Stadium',
                'Zayed Cricket Stadium, Abu Dhabi': 'Sheikh Zayed Stadium'}
    df['Venue'] = df['Venue'].replace(venues_dict)
    # Two teams have changed their name over the period the dataset covers
    team_name_changes = {'Kings XI Punjab': 'Punjab Kings',
                        'Delhi Daredevils': 'Delhi Capitals'}
    df['Team1'] = df['Team1'].replace(team_name_changes)
    # Updating the team name for Rising Pune Supergiant
    team_name_dict = {'Rising Pune Supergiants': 'Rising Pune Supergiant'}
    df['Team1'] = df['Team1'].replace(team_name_dict)
    # Updating the other team name columns
    df['Team2'] = df['Team2'].replace(team_name_changes)
    df['TossWinner'] = df['TossWinner'].replace(team_name_changes)
    df['WinningTeam'] = df['WinningTeam'].replace(team_name_changes)
    df['Team2'] = df['Team2'].replace(team_name_dict)
    df['TossWinner'] = df['TossWinner'].replace(team_name_dict)
    df['WinningTeam'] = df['WinningTeam'].replace(team_name_dict)
    # Identifying and removing 4 "NoResult" Values
    df['WinningTeam'] = df['WinningTeam'].fillna('NoResults')
    df['Player_of_Match'] = df['Player_of_Match'].fillna('NoResults')
    df['SuperOver'] = df['SuperOver'].fillna('NoResults')
    df.loc[df['WinningTeam'].isna(), 'method'] = 'NoResults'
    df.loc[df['WinningTeam'].isna(), 'Margin'] = 0
    # The method column held no useful information and was dropped
    df = df.drop(columns='method')
    # Results from margin related to Superover and have been filled with 0
    df['Margin'] = df['Margin'].fillna(0)

    # Updating dates so we are able to identify seasonal changes in data
    season_dict = {'2020/21': '2020',
                '2009/10': '2010',
                '2007/08': '2008'}

    df['Season'] = df['Season'].replace(season_dict)
    # Selecting columns to strip
    df_columns = list(df.select_dtypes(include='object').columns)
    # Stripping the data for each column
    for column in df_columns:
        df[column] = [text.strip().lower() for text in df[column]]

    df['Team1Players'] = df['Team1Players'].apply(lambda x: ast.literal_eval(x))
    df['Team2Players'] = df['Team2Players'].apply(lambda x: ast.literal_eval(x))

    print("✅ data cleaned")
    return df


def feature_engineer(df: pd.DataFrame) -> pd.DataFrame:
    '''
    This is where we create the new features for the model.
    They should be:
    - team_1_win --> 1 or 0 if team 1 one the match
    - innings_total --> to be used to calculate batting avgs etc.
    - team_1_batting_average --> the average amount of runs that team1 scores in a match
    - team_2_batting_average --> the average amount of runs that team2 scores in a match
    - team_1_toss_winner --> 1 or 0 if team1 won the toss
    - match_importance --> ordinal value given to the importance of the match. Final or regular etc.
    - team_1_points_against_avg --> the average number of point opposition teams score agains team_1
    - team_2_points_against_avg --> the average number of point opposition teams score agains team_2
    - team_1_avg_mvp --> average number of times that team_1 has MVP
    - team_2_avg_mvp --> average number of times that team_2 has MVP
    '''
    # create the binary column Team1_Win
    df['Team1_Win'] = (df['Team1'] == df['WinningTeam']).astype(int)
    # create the innings total
    df['innings_total'] = df.groupby(['ID', 'innings'])['total_run'].transform('sum')
    df = df[(df['innings'] == 1) | (df['innings'] == 2)]
    # Convert the innings totals into columns

    # Group by 'ID' and 'innings' and calculate innings_total
    innings_totals = df.groupby(['ID', 'innings'])['innings_total'].mean().unstack()
    innings_totals.columns = ['Team1_innings_total', 'Team2_innings_total']

    # Merge the innings_totals DataFrame back into new_df on 'ID'
    df = pd.merge(df, innings_totals, left_on='ID', right_index=True)
    df['team_batting_average'] = df.groupby('BattingTeam')['total_run'].transform('sum') / df.groupby('BattingTeam')['ID'].transform('nunique')
    # Extract the relevant columns
    selected_columns = ['ID', 'innings', 'Team1', 'Team2', 'team_batting_average', 'MatchNumber','WinningTeam', 'innings_total', 'City', 'Date', 'Venue', 'TossWinner', 'TossDecision', 'Team1Players', 'Team2Players']
    new_df = df[selected_columns].copy()
    new_df.head()

    # Filter innings 1 (Team1's innings)
    team_a_data = new_df[new_df['innings'] == 1][['ID','Team1', 'team_batting_average' ]]
    team_a_data.rename(columns={'team_batting_average': 'Team_1_batting_average'}, inplace=True)

    # Filter innings 2 (Team1's innings)
    team_b_data = new_df[new_df['innings'] == 2][['ID', 'Team2', 'team_batting_average']]
    team_b_data.rename(columns={'team_batting_average': 'Team_2_batting_average'}, inplace=True)

    # Merge the data from innings 1 and innings 2 for each match ID
    merged_data = pd.merge(team_a_data, team_b_data, on='ID')
    # Drop duplicates based on 'ID' column and keep the first occurrence
    final_data = merged_data.drop_duplicates(subset='ID', keep='first')
    # Reset the index
    final_data.reset_index(drop=True, inplace=True)
    # drop the duplicate columns
    final_data.drop(columns=["Team1", "Team2"], inplace=True)
    # merge with the main df
    df = pd.merge(df, final_data, on = "ID")
    # drop the useless column
    df.drop(columns=['team_batting_average'], inplace=True)
    # new binary column for the toss winner
    df['team_1_toss_winner'] = (df['Team1'] == df['TossWinner']).astype(int)

    # Apply the function
    df['MatchImportance'] = df['MatchNumber'].apply(map_match_number)
    # Calculate the average points scored against a each team
    df['Team1_points_against_avg'] = df.groupby('Team1')['Team2_innings_total'].transform('mean')
    df['Team2_points_against_avg'] = df.groupby('Team2')['Team1_innings_total'].transform('mean')
    # Calculate the average number of times that team_1 has MVP

    df['Team_MVP'] = df.apply(lambda row: get_match_winner(row['Player_of_Match'], row['Team1Players'], row['Team2Players']), axis=1)

    df['Team_MVP'] = df.apply(replace_team_mvp_with_name, axis=1)
    team_mvp_counts = df['Team_MVP'].value_counts().reset_index()

    team_mvp_counts.columns = ['Team_MVP', 'MVP_Count']

    # Create the columns Team1_MVP_appearances e Team2_MVP_appearances
    df = df.merge(team_mvp_counts, left_on='Team1', right_on='Team_MVP', how='left').fillna(0)
    df.rename(columns={'MVP_Count': 'Team1_MVP_appearances'}, inplace=True)

    df = df.merge(team_mvp_counts, left_on='Team2', right_on='Team_MVP', how='left').fillna(0)
    df.rename(columns={'MVP_Count': 'Team2_MVP_appearances'}, inplace=True)

    # Drop columns
    df.drop(['Team_MVP_x', 'Team_MVP_y', 'Team_MVP'], axis=1, inplace=True)

    # Now, let's create the average of MVP appearences
    total_games_team1 = df['Team1'].value_counts()
    total_games_team2 = df['Team2'].value_counts()

    total_games = total_games_team2 + total_games_team1

    df['Team1_MVP_average'] = df['Team1_MVP_appearances'] / total_games[df['Team1']].values
    df['Team2_MVP_average'] = df['Team2_MVP_appearances'] / total_games[df['Team2']].values
    df = df.drop(columns = ['Team1_MVP_appearances', 'Team2_MVP_appearances'])

    # Drop duplicates based on 'ID' column and keep the first occurrence
    final_data = df.drop_duplicates(subset='ID', keep='first')

    print(f"✅ New featured engineered new shape is {final_data.shape}")

    return final_data

def get_data_with_cache(
        gcp_project:str,
        query:str,
        cache_path:Path,
        data_has_header=True
    ) -> pd.DataFrame:


    #Retrieve `query` data from BigQuery, or from `cache_path` if the file exists
    #Store at `cache_path` if retrieved from BigQuery for future use


    if cache_path.is_file():
        print("\nLoad data from local CSV...")
        df = pd.read_csv(cache_path, header='infer' if data_has_header else None)
    else:
        print("\nLoad data from BigQuery server...")
        client = bigquery.Client(project=gcp_project)
        query_job = client.query(query)
        result = query_job.result()
        df = result.to_dataframe()

        # Store as CSV if the BQ query returned at least one valid line
        if df.shape[0] > 1:
            df.to_csv(cache_path, header=data_has_header, index=False)

    print(f"✅ Data loaded, with shape {df.shape}")

    return df

def load_data_to_bq(
        data: pd.DataFrame,
        gcp_project:str,
        bq_dataset:str,
        table: str,
        truncate: bool
    ) -> None:

    #- Save the DataFrame to BigQuery
    #- Empty the table beforehand if `truncate` is True, append otherwise


    assert isinstance(data, pd.DataFrame)
    full_table_name = f"{gcp_project}.{bq_dataset}.{table}"
    print(f"\nSave data to BigQuery @ {full_table_name}...:")

    # Load data onto full_table_name
    data.columns = [f"_{column}" if not str(column)[0].isalpha() and not str(column)[0] == "_" else str(column) for column in data.columns]

    client = bigquery.Client()

    # Define write mode and schema
    write_mode = "WRITE_TRUNCATE" if truncate else "WRITE_APPEND"
    job_config = bigquery.LoadJobConfig(write_disposition=write_mode)

    print(f"\n{'Write' if truncate else 'Append'} {full_table_name} ({data.shape[0]} rows)")

    # Load data
    job = client.load_table_from_dataframe(data, full_table_name, job_config=job_config)
    result = job.result()  # wait for the job to complete

    print(f"✅ Data saved to bigquery, with shape {data.shape}")

'''
IGNORE THIS FOR NOW

def get_data(userinput: list) -> pd.DataFrame:

    Here is where the data is taken. This could be based on the user input
    on Streamlit. The information could be taken from BigQuery once the user
    asks for a prediction.

    pass # ADD CODE HERE
'''

# define the importance of each match
def map_match_number(value):
            if isinstance(value, int) or value.isnumeric():
                return 1
            elif value in ['qualifier 2', 'eliminator', 'qualifier 1', 'qualifier', 'elimination final', '3rd place play-off', 'semi final']:
                return 2
            elif value == 'final':
                return 3
            else:
                return 0

def get_match_winner(player_of_match, team1_players, team2_players):
            if pd.isna(player_of_match):
                return 'N/A'

            if pd.isna(team1_players):
                team1_players = []

            if pd.isna(team2_players):
                team2_players = []

            if player_of_match in team1_players:
                return 'Team1'
            elif player_of_match in team2_players:
                return 'Team2'
            else:
                return 'N/A'

def replace_team_mvp_with_name(row):
            if row['Team_MVP'] == 'Team1':
                return row['Team1']
            elif row['Team_MVP'] == 'Team2':
                return row['Team2']
            else:
                return row['Team_MVP']"""


'import numpy as np\nimport pandas as pd\nimport ast\n#from google.cloud import bigquery\nfrom pathlib import Path\n\n\'\'\'\nThis module is used for cleaning data that can be\nbe used to pass into the model in the model.py function\n\'\'\'\n\ndef clean_data(df: pd.DataFrame) -> pd.DataFrame:\n    \'\'\'\n    Clean data by:\n    - Removing or imputing NaN\n    \'\'\'\n    ## cleaning data from ball_by_ball dataframe\n    # drop duplicates\n    df = df.drop_duplicates()\n    # assign the good value to overs\n    df["overs"] = df["overs"] + 1\n    # cleaning the string values\n    # batter\n    df["batter"] = [text.strip() for text in df["batter"]] # removing space\n    df["batter"] = [text.lower() for text in df["batter"]] # lowercase\n    # bowler\n    df["bowler"] = [text.strip() for text in df["bowler"]]\n    df["bowler"] = [text.lower() for text in df["bowler"]]\n    # non-striker\n    df.rename(columns={"non-striker": "non_striker"}, inplace=True) #renaming the non_striker\n    df[

# Feature Engineer

In [115]:
def player_batting_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Takes the ball by ball match cleaned dataframe as a base.
    Creates a dataframe of player batting features extracted from the
    ball by ball match dataframe.
    Returns the features processed.
    """
    assert isinstance(df, pd.DataFrame)

    df = df[PLAYER_PROCESSING_COLUMNS]

    ### Total Runs
    batter_total_runs = df.groupby(['batter'], as_index=False)['batsman_run']\
        .sum('batsman_run').rename(columns={'batsman_run': 'total_runs'})

    ### Balls Faced
    balls_faced = df[df['extra_type'].isin(['noextra', 'legbyes', 'byes', 'noballs'])]\
                .groupby(['batter'], as_index=False)['ballnumber'].count()\
                .rename(columns={'ballnumber': 'balls_faced'})

    ### Batter Innings
    batter_innings = df.groupby(['batter'], as_index=False)['ID'].nunique()\
                    .rename(columns={'ID': 'batter_innings'})
    non_striker_innings = df.groupby(['non_striker'], as_index=False)['ID'].nunique()\
                    .rename(columns={'ID': 'non_striker_innings', 'non_striker': 'batter'})
    combined_innings_df = batter_innings.merge(non_striker_innings, \
                    on='batter', how='outer').fillna(0)
    combined_innings_df['bat_innings'] = combined_innings_df[['batter_innings', 'non_striker_innings']].max(axis=1)
    bat_innings = combined_innings_df.drop(columns=['batter_innings', 'non_striker_innings'])

    ### Boundaries
    if len(df[df['non_boundary'] == 1]) > 0:
        non_boundary = list(df[df['non_boundary'] == 1].index)
        temp_df = df.drop(index=non_boundary)
    else:
        temp_df = df
    fours = temp_df[temp_df['batsman_run'] == 4].groupby(['batter'], as_index=False)\
            ['batsman_run'].count().rename(columns={'batsman_run': 'fours'})
    sixes = temp_df[temp_df['batsman_run'] == 6].groupby(['batter'], as_index=False)\
            ['batsman_run'].count().rename(columns={'batsman_run': 'sixes'})

    ### Batsman Totals
    batsman_score_game = df.groupby(['ID', 'batter'], as_index=False)\
                        ['batsman_run'].sum().drop(columns='ID')
    fifty = batsman_score_game[batsman_score_game['batsman_run'] >= 50]\
        .groupby(['batter'], as_index=False).count().rename(columns={'batsman_run': '50s'})
    hundred = batsman_score_game[batsman_score_game['batsman_run'] >= 100]\
        .groupby(['batter'], as_index=False).count().rename(columns={'batsman_run': '100s'})
    merged_totals = fifty.merge(hundred, on='batter', how='outer').fillna(0)
    merged_totals['50s'] = merged_totals['50s'].sub(merged_totals['100s']).astype('int')
    merged_totals['100s'] = merged_totals['100s'].astype('int')

    ### Batsman Not Out
    last_ball_index_temp = df.groupby(['ID', 'innings'], as_index=False)['innings']\
                        .idxmax().sort_values(by='innings', ascending=False)['innings'].tolist()
    last_ball_index_temp = last_ball_index_temp[:-1]
    last_ball_index = [index - 1 for index in last_ball_index_temp]
    last_ball_index.insert(0, df.shape[0])
    player_out_final_ball = df[(df['isWicketDelivery']==1) & (df.index.isin(last_ball_index))]\
                            .groupby(['player_out'], as_index=False)['ID'].count()\
                            .rename(columns={'ID': 'out_count', 'player_out': 'batter'})
    final_ball_batter = df[df.index.isin(last_ball_index)].groupby(['batter'], as_index=False)\
                        ['ID'].count().rename(columns={'ID': 'at_bat_count'})
    final_ball_non_striker = df[df.index.isin(last_ball_index)].groupby(['non_striker'], as_index=False)\
                            ['ID'].count().rename(columns={'ID': 'non_striker_count', 'non_striker': 'batter'})
    combined_final_ball = final_ball_batter.merge(final_ball_non_striker, on='batter', how='outer')\
                            .merge(player_out_final_ball, on='batter', how='outer')
    combined_final_ball.fillna(0, inplace=True)
    combined_final_ball['not_out'] = combined_final_ball['at_bat_count']\
                                    .add(combined_final_ball['non_striker_count'])\
                                    .sub(combined_final_ball['out_count']).astype('int')
    not_out = combined_final_ball[['batter', 'not_out']]

    ### High Score
    high_score = df.groupby(['ID', 'batter'], as_index=False)['batsman_run']\
                        .sum().groupby('batter', as_index=False)['batsman_run'].max()\
                        .rename(columns={'batsman_run': 'high_score'})

    ### Merge
    batting_stats_merged = bat_innings.merge(batter_total_runs, on='batter', how='outer')\
                        .merge(not_out, on='batter', how='outer')\
                        .merge(merged_totals, on='batter', how='outer')\
                        .merge(fours, on='batter', how='outer')\
                        .merge(sixes, on='batter', how='outer')\
                        .merge(high_score, on='batter', how='outer')\
                        .merge(balls_faced, on='batter', how='outer').fillna(0)

    ### Feature creation
    batting_stats_merged['average_score'] = round\
        ((batting_stats_merged['total_runs'] / batting_stats_merged['bat_innings']), 2).fillna(0)
    batting_stats_merged['batting_average'] = round((batting_stats_merged['total_runs'] / \
        (batting_stats_merged['bat_innings'] - batting_stats_merged['not_out'])), 2).fillna(0)
    batting_stats_merged['batting_average'] = np.where(batting_stats_merged\
        ['batting_average'] == np.inf, batting_stats_merged['average_score'], batting_stats_merged['batting_average'])
    batting_stats_merged['batting_strike_rate'] = round\
        ((batting_stats_merged['total_runs'] * 100)/batting_stats_merged['balls_faced'], 2).fillna(0)

    batting_stats_merged = batting_stats_merged.rename(columns={'batter': 'player'})

    print("✅ Batting Features Created")

    return batting_stats_merged


def player_bowling_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Takes the ball by ball match cleaned dataframe as a base.
    Creates a dataframe of player bowling features extracted from the
    ball by ball match dataframe.
    Returns the features processed.
    """
    assert isinstance(df, pd.DataFrame)

    df = df[PLAYER_PROCESSING_COLUMNS]

    ### Bowler Innings
    bowler_innings = df.groupby(['bowler'], as_index=False)['ID'].nunique()\
                    .rename(columns={'ID': 'bowler_innings'})

    ### Balls Bowled
    balls_bowled = df[df['extra_type'].isin(['noextra', 'legbyes', 'byes', 'penalty'])]\
                .groupby(['bowler'], as_index=False)['ballnumber'].count()\
                .rename(columns={'ballnumber': 'balls_bowled'})

    ### Runs Conceded
    runs_conceded = df[df['extra_type'].isin(['noextra', 'wides', 'noballs'])]\
                .groupby(['bowler'], as_index=False)['total_run'].sum()\
                .rename(columns={'total_run': 'runs_conceded'})

    ### Wickets Taken
    wickets = df[(df['isWicketDelivery'] == 1) & (df['kind'].isin\
                (['nokind', 'caught', 'caught and bowled', 'bowled','stumped', 'lbw', 'hit wicket']))]\
                .groupby(['bowler'], as_index=False)['isWicketDelivery'].count()\
                .rename(columns={'isWicketDelivery': 'wickets'})

    ### Wickets Per Game Stats
    wicket_count_per_game = df[(df['isWicketDelivery'] == 1) & (df['kind'].isin\
                (['nokind', 'caught', 'caught and bowled', 'bowled','stumped', 'lbw', 'hit wicket']))]\
                .groupby(['ID', 'bowler'], as_index=False)['isWicketDelivery'].count()\
                .rename(columns={'isWicketDelivery': 'wickets'})

    four_wickets = wicket_count_per_game[wicket_count_per_game['wickets'] == 4]\
                .groupby(['bowler'], as_index=False)['wickets'].count()\
                .rename(columns={'wickets': 'four_wickets'})

    five_wickets = wicket_count_per_game[wicket_count_per_game['wickets'] == 5]\
                .groupby(['bowler'], as_index=False)['wickets'].count()\
                .rename(columns={'wickets': 'five_wickets'})

    ### Merge
    bowling_stats_merged = bowler_innings.merge(balls_bowled, on='bowler', how='outer')\
                        .merge(runs_conceded, on='bowler', how='outer')\
                        .merge(wickets, on='bowler', how='outer')\
                        .merge(four_wickets, on='bowler', how='outer')\
                        .merge(five_wickets, on='bowler', how='outer').fillna(0)

    ### Feature Creation
    bowling_stats_merged['bowling_average'] = round\
        ((bowling_stats_merged['runs_conceded'] / bowling_stats_merged['wickets']), 2).fillna(0)
    bowling_stats_merged['bowling_average'] = np.where\
        (bowling_stats_merged['bowling_average'] == np.inf, 0, bowling_stats_merged['bowling_average'])
    bowling_stats_merged['bowling_economy_rate'] = round\
        ((bowling_stats_merged['runs_conceded']/(bowling_stats_merged['balls_bowled'] / 6)), 2)
    bowling_stats_merged['bowling_strike_rate'] = round((bowling_stats_merged['balls_bowled']/bowling_stats_merged['wickets']), 2)
    bowling_stats_merged['bowling_strike_rate'] = np.where\
        (bowling_stats_merged['bowling_strike_rate'] == np.inf, 0, bowling_stats_merged['bowling_strike_rate'])

    bowling_stats_merged = bowling_stats_merged.rename(columns={'bowler': 'player'})

    print("✅ Bowling Features Created")

    return bowling_stats_merged

def win_loss_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Takes a cleaned dataframe as a base.
    Creates a dataframe of player win loss statistics.
    Returns the features processed.
    """
    assert isinstance(df, pd.DataFrame)

    df = df[WIN_LOSS_PROCESSING_COLUMNS]

    ### Group to remove duplicates
    df = df.groupby('ID', as_index=False).first()

    ### Team 1 Wins
    def team_1_wins(Team1, Team2, WinningTeam):
        if WinningTeam == Team1:
            return 1
        elif WinningTeam == Team2:
            return 0
        else:
            return -1

    df['team_1_win'] = df.apply(lambda row: \
        team_1_wins(row['Team1'], row['Team2'], row['WinningTeam']), axis=1)

    ### Winning Player Count
    def winning_team_players(team_1_win, Team1Players, Team2Players):
        if team_1_win == 1:
            return Team1Players
        elif team_1_win == 0:
            return Team2Players
        else:
            return []

    df['winning_team_players'] = df.apply(lambda row: winning_team_players\
        (row['team_1_win'], row['Team1Players'], row['Team2Players']), axis=1)

    player_wins = []
    def player_wins_list(winning_team_players):
        for row in winning_team_players:
            for player in row:
                player_wins.append(player)
        return player_wins

    player_wins_list(df['winning_team_players'])

    player_win_count = {}
    for player in player_wins:
        if player in player_win_count:
            player_win_count[player] += 1
        else:
            player_win_count[player] = 1

    wins = pd.DataFrame({'player': player_win_count.keys(), \
                        'wins': player_win_count.values()})

    ### Losing Player Count
    def losing_team_players(team_1_win, Team1Players, Team2Players):
        if team_1_win == 0:
            return Team1Players
        elif team_1_win == 1:
            return Team2Players
        else:
            return []

    df['losing_team_players'] = df.apply(lambda row: losing_team_players\
        (row['team_1_win'], row['Team1Players'], row['Team2Players']), axis=1)

    player_loses = []
    def player_loses_list(losing_team_players):
        for row in losing_team_players:
            for player in row:
                player_loses.append(player)
        return player_loses

    player_loses_list(df['losing_team_players'])

    player_loss_count = {}
    for player in player_loses:
        if player in player_loss_count:
            player_loss_count[player] += 1
        else:
            player_loss_count[player] = 1

    loses = pd.DataFrame({'player': player_loss_count.keys(), \
                            'loses': player_loss_count.values()})

    ### Matches
    matches = {}

    for player in player_win_count:
        if player in player_loss_count:
            matches[player] = player_win_count[player] + player_loss_count[player]
        elif player not in player_loss_count:
            matches[player] = player_win_count[player]

    for player in player_loss_count:
        if player not in player_win_count:
            matches[player] = player_loss_count[player]

    matches = pd.DataFrame({'player': matches.keys(), \
                            'matches': matches.values()})

    ### Merge
    win_loss_merge = matches.merge(wins, on='player', how='outer')\
                        .merge(loses, on='player', how='outer').fillna(0)

    ### Create Win Ratio
    win_loss_merge['win_ratio'] = \
        round((win_loss_merge['wins']/win_loss_merge['matches']), 2)

    print("✅ Win Loss Features Created")

    return win_loss_merge

def player_features_dataset(df):
    """
    Compiles the final player dataset.
    """
    assert isinstance(df, pd.DataFrame)

    batting = player_batting_features(df)
    bowling = player_bowling_features(df)
    win_loss = win_loss_features(df)

    final_player_dataset = batting.merge(bowling, on='player', how='outer')\
                            .merge(win_loss, on='player', how='outer').fillna(0)

    print("✅ Final Player Dataset Created")

    return final_player_dataset

def feature_engineer(df: pd.DataFrame) -> pd.DataFrame:
    '''
    This is where we create the new features for the model combined with the engineered
    player features.
    '''

    players_df = player_features_dataset(df)

    #### NEW FEATURES ####
    # Calculates Team1 and Team2 weighted average
    df['Avg_Weighted_Score_Team1'] = df['Team1Players'].apply(lambda players: weighted_average_score(players, players_df))
    df['Avg_Weighted_Score_Team2'] = df['Team2Players'].apply(lambda players: weighted_average_score(players, players_df))

    df['batting_average_PlayersTeam1_weighted'] = df['Team1Players'].apply(lambda players: weighted_batting_average(players, players_df))
    df['batting_average_PlayersTeam2_weighted'] = df['Team2Players'].apply(lambda players: weighted_batting_average(players, players_df))

    df['batting_strike_rate_PlayersTeam1_weighted'] = df['Team1Players'].apply(lambda players: weighted_batting_strike_rate(players, players_df))
    df['batting_strike_rate_PlayersTeam2_weighted'] = df['Team2Players'].apply(lambda players: weighted_batting_strike_rate(players, players_df))

    # Calculates the bowling weighted averages
    df['bowling_average_PlayersTeam1'] = df['Team1Players'].apply(lambda players: players_df[players_df['player'].isin(players)]['bowling_average'].mean())
    df['bowling_average_PlayersTeam2'] = df['Team2Players'].apply(lambda players: players_df[players_df['player'].isin(players)]['bowling_average'].mean())

    df['bowling_economy_rate_PlayersTeam1'] = df['Team1Players'].apply(lambda players: players_df[players_df['player'].isin(players)]['bowling_economy_rate'].mean())
    df['bowling_economy_rate_PlayersTeam2'] = df['Team2Players'].apply(lambda players: players_df[players_df['player'].isin(players)]['bowling_economy_rate'].mean())

    df['bowling_strike_rate_PlayersTeam1'] = df['Team1Players'].apply(lambda players: players_df[players_df['player'].isin(players)]['bowling_strike_rate'].mean())
    df['bowling_strike_rate_PlayersTeam2'] = df['Team2Players'].apply(lambda players: players_df[players_df['player'].isin(players)]['bowling_strike_rate'].mean())

    df['win_ratio_PlayersTeam1'] = df['Team1Players'].apply(lambda players: players_df[players_df['player'].isin(players)]['win_ratio'].mean())
    df['win_ratio_PlayersTeam2'] = df['Team2Players'].apply(lambda players: players_df[players_df['player'].isin(players)]['win_ratio'].mean())

    # Calculates the differences between Team1 and Team2
    df['Avg_Weighted_Score_diff'] = df['Avg_Weighted_Score_Team1'] - df['Avg_Weighted_Score_Team2']

    df['batting_average_weighted_diff'] = df['batting_average_PlayersTeam1_weighted'] - df['batting_average_PlayersTeam2_weighted']

    df['batting_strike_rate_weighted_diff'] = df['batting_strike_rate_PlayersTeam1_weighted'] - df['batting_strike_rate_PlayersTeam2_weighted']

    df['bowling_average_diff'] = df['bowling_average_PlayersTeam1'] - df['bowling_average_PlayersTeam2']

    df['bowling_economy_rate_diff'] = df['bowling_economy_rate_PlayersTeam1'] - df['bowling_economy_rate_PlayersTeam2']

    df['bowling_strike_rate_diff'] = df['bowling_strike_rate_PlayersTeam1'] - df['bowling_strike_rate_PlayersTeam2']

    df['win_ratio_diff'] = df['win_ratio_PlayersTeam1'] - df['win_ratio_PlayersTeam2']

    # Drop duplicates based on 'ID' column and keep the first occurrence
    final_data = df.drop_duplicates(subset='ID', keep='first')

    print(f"✅ New featured engineered new shape is {final_data.shape}")

    return final_data

def weighted_average_score(players_list, players_df):
    # Filters the player information in the DataFrame based on the names of the players in the list
    player_info = players_df[players_df['player'].isin(players_list)].copy()

    # Calculates the weight for each player based on the average_score (for example, using the square root function)

    #square root function:
    player_info['weight'] = np.sqrt(player_info['average_score'])

    # Calculates the weighted average using the weights
    weighted_avg = (players_df['average_score'] * player_info['weight']).sum() / player_info['weight'].sum()

    return weighted_avg

def weighted_batting_average(players_list, players_df):

    player_info = players_df[players_df['player'].isin(players_list)].copy()

    player_info['weight'] = np.sqrt(player_info['average_score'])

    weighted_avg = (players_df['batting_average'] * player_info['weight']).sum() / player_info['weight'].sum()

    return weighted_avg

def weighted_batting_strike_rate(players_list, players_df):

    player_info = players_df[players_df['player'].isin(players_list)].copy()

    player_info['weight'] = np.sqrt(player_info['average_score'])

    weighted_avg = (players_df['batting_strike_rate'] * player_info['weight']).sum() / player_info['weight'].sum()

    return weighted_avg

# define the importance of each match
def map_match_number(value):
            if isinstance(value, int) or value.isnumeric():
                return 1
            elif value in ['qualifier 2', 'eliminator', 'qualifier 1', 'qualifier', 'elimination final', '3rd place play-off', 'semi final']:
                return 2
            elif value == 'final':
                return 3
            else:
                return 0

def get_match_winner(player_of_match, team1_players, team2_players):
            if pd.isna(player_of_match):
                return 'N/A'

            if pd.isna(team1_players):
                team1_players = []

            if pd.isna(team2_players):
                team2_players = []

            if player_of_match in team1_players:
                return 'Team1'
            elif player_of_match in team2_players:
                return 'Team2'
            else:
                return 'N/A'

def replace_team_mvp_with_name(row):
            if row['Team_MVP'] == 'Team1':
                return row['Team1']
            elif row['Team_MVP'] == 'Team2':
                return row['Team2']
            else:
                return row['Team_MVP']


# Params

In [18]:


##################  CONSTANTS  ##################
PLAYER_PROCESSING_COLUMNS = ['ID', 'innings', 'overs', 'ballnumber', 'batter',
        'bowler', 'non_striker', 'extra_type', 'batsman_run', 'extras_run',
        'total_run', 'non_boundary', 'isWicketDelivery', 'player_out', 'kind',
        'fielders_involved', 'BattingTeam']

WIN_LOSS_PROCESSING_COLUMNS = ['ID', 'Season', 'Team1', 'Team2', 'WinningTeam',
                             'Player_of_Match', 'Team1Players', 'Team2Players']

COLUMN_NAMES = ['City', 'Season', 'Team1', 'Team2', 'Venue', 'TossWinner',
       'TossDecision', 'WinningTeam', 'Avg_Weighted_Score_diff',
       'batting_average_weighted_diff', 'batting_strike_rate_weighted_diff',
       'bowling_average_diff', 'bowling_economy_rate_diff', 'win_ratio_diff']

TARGET_NAME = ['WinningTeam']

FEATURE_NAMES = ['City', 'Season', 'Team1', 'Team2', 'Venue', 'TossWinner',
       'TossDecision', 'Avg_Weighted_Score_diff',
       'batting_average_weighted_diff', 'batting_strike_rate_weighted_diff',
       'bowling_average_diff', 'bowling_economy_rate_diff', 'win_ratio_diff']

CATAGORICAL_COLUMNS = ['Team1', 'Team2', 'Venue', 'TossDecision', 'City']

NUMERICAL_COLUMNS = ['TossWinner', 'TossDecision', 'Avg_Weighted_Score_diff',
       'batting_average_weighted_diff', 'batting_strike_rate_weighted_diff',
       'bowling_average_diff', 'bowling_economy_rate_diff', 'win_ratio_diff']

TEAM_NAMES = ['rajasthan royals', 'royal challengers bangalore',
       'sunrisers hyderabad', 'delhi capitals', 'chennai super kings',
       'gujarat titans', 'lucknow super giants', 'kolkata knight riders',
       'punjab kings', 'mumbai indians', 'rising pune supergiant',
       'gujarat lions', 'pune warriors', 'deccan chargers',
       'kochi tuskers kerala']

##################  CONSTANTS  ##################
PLAYER_PROCESSING_COLUMNS = ['ID', 'innings', 'overs', 'ballnumber', 'batter',
        'bowler', 'non_striker', 'extra_type', 'batsman_run', 'extras_run',
        'total_run', 'non_boundary', 'isWicketDelivery', 'player_out', 'kind',
        'fielders_involved', 'BattingTeam']

WIN_LOSS_PROCESSING_COLUMNS = ['ID', 'Season', 'Team1', 'Team2', 'WinningTeam',
                             'Player_of_Match', 'Team1Players', 'Team2Players']

# Game Average

In [112]:
def match_filter(df: pd.DataFrame,
               number_of_matches: int = "all",
               team_1: list = "all",
               team_2: list = "all",
               season: list = "all") -> pd.DataFrame:
    """
    Takes a cleaned dataframe as a base.
    Filters the dataframe by number of targets:

    number_of_matches, the maximum number of matches
    team_1, the list of teams to include as team_1
    team_2, the list of teams to include as team_2
    season, the list of seasons to include in the filter.
    Returns a filtered dataframe
    """

    if team_1 != "all":
        df = df[df['Team1'].isin(team_1)]

    if team_2 != "all":
        df = df[df['Team2'].isin(team_2)]

    if season != "all":
        df = df[df['Season'].isin(season)]

    if number_of_matches != "all":
        match_id = df['ID'].unique().tolist()
        df = df[df['ID'].isin(match_id[0:number_of_matches])]

    return df


def player_team_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Takes a cleaned dataframe as a base.

    Processes the dataframe to output player data using the feature engineer
    module

    Returns a dataframe for both team_1 and team_2 players
    """

    match_id = df['ID'].unique().tolist()

    team_1_list = []
    team_2_list = []

    for match in match_id:
        match_df = df[df['ID']==match]

        team_1_players = match_df['Team1Players'].iloc[0]
        team_2_players = match_df['Team2Players'].iloc[0]

        player_match_df = player_features_dataset(match_df)

        t1 = player_match_df[player_match_df['player'].isin(team_1_players)]
        t2 = player_match_df[player_match_df['player'].isin(team_2_players)]

        t1.loc[:, 'ID'] = match
        t2.loc[:, 'ID'] = match

        team_1_list.append(t1)
        team_2_list.append(t2)

    team_1 = pd.concat(team_1_list, ignore_index=True)
    team_2 = pd.concat(team_2_list, ignore_index=True)

    return team_1, team_2


def team_average(df_filtered: pd.DataFrame,
                 team_1: pd.DataFrame,
                 team_2: pd.DataFrame) -> pd.DataFrame:
    """
    Takes the filtered dataframe from match_filter()
    and team_1, team_2 dataframe from player_team_data()

    Processes the dataframe to output a combined dataframe
    with average scores for each team grouped by the match ID

    Returns a combined dataframe.
    """

    df = df_filtered[WIN_LOSS_PROCESSING_COLUMNS].groupby('ID').first()
    columns = team_1.drop(columns='player').columns

    for column in columns:
        df[f"Team_1_{column}"] = team_1.groupby('ID')[column].mean()
        df[f"Team_2_{column}"] = team_2.groupby('ID')[column].mean()

    return df


def game_average(df: pd.DataFrame,
                number_of_matches: int = "all",
                team_1: list = "all",
                team_2: list = "all",
                season: list = "all") -> pd.DataFrame:
    """
    Takes a cleaned dataframe as a base.

    A combined process for returning the average match data for a filtered
    dataframe using:
    match_filter()
    player_team_data()
    team_average()

    number_of_matches, the maximum number of matches
    team_1, the list of teams to include as team_1
    team_2, the list of teams to include as team_2
    season, the list of seasons to include in the filter.

    Returns a dataframe
    """

    df_filtered = match_filter(df, number_of_matches, team_1, team_2, season)

    team_1_df, team_2_df = player_team_data(df_filtered)

    average_df = team_average(df_filtered, team_1_df, team_2_df)

    return average_df


In [5]:
ball_df = pd.read_csv('~/code/patrickevans29/raw_data/IPL_Ball_by_Ball_2008_2022.csv')
match_df = pd.read_csv('~/code/patrickevans29/raw_data/IPL_Matches_2008_2022.csv')

In [7]:
df = ball_df.merge(match_df, on='ID', how='outer')

In [9]:
clean_df = clean_data(df)

✅ data cleaned


In [23]:
teams = ['rajasthan royals', 'royal challengers bangalore',
       'sunrisers hyderabad', 'delhi capitals', 'chennai super kings',
       'gujarat titans', 'lucknow super giants', 'kolkata knight riders',
       'punjab kings', 'mumbai indians']

In [113]:
game_average_df = game_average(clean_df, team_1=teams, team_2=teams)

✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player

/var/folders/4n/3hw1gckd4zg3786ftn8gpp8c0000gn/T/ipykernel_37759/1567032424.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1.loc[:, 'ID'] = match
/var/folders/4n/3hw1gckd4zg3786ftn8gpp8c0000gn/T/ipykernel_37759/1567032424.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t2.loc[:, 'ID'] = match
/var/folders/4n/3hw1gckd4zg3786ftn8gpp8c0000gn/T/ipykernel_37759/1567032424.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player

/var/folders/4n/3hw1gckd4zg3786ftn8gpp8c0000gn/T/ipykernel_37759/1567032424.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1.loc[:, 'ID'] = match
/var/folders/4n/3hw1gckd4zg3786ftn8gpp8c0000gn/T/ipykernel_37759/1567032424.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t2.loc[:, 'ID'] = match
/var/folders/4n/3hw1gckd4zg3786ftn8gpp8c0000gn/T/ipykernel_37759/1567032424.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player

...

✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created


/var/folders/4n/3hw1gckd4zg3786ftn8gpp8c0000gn/T/ipykernel_37759/1567032424.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1.loc[:, 'ID'] = match
/var/folders/4n/3hw1gckd4zg3786ftn8gpp8c0000gn/T/ipykernel_37759/1567032424.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t2.loc[:, 'ID'] = match
/var/folders/4n/3hw1gckd4zg3786ftn8gpp8c0000gn/T/ipykernel_37759/1567032424.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created


/var/folders/4n/3hw1gckd4zg3786ftn8gpp8c0000gn/T/ipykernel_37759/1567032424.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1.loc[:, 'ID'] = match
/var/folders/4n/3hw1gckd4zg3786ftn8gpp8c0000gn/T/ipykernel_37759/1567032424.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t2.loc[:, 'ID'] = match
/var/folders/4n/3hw1gckd4zg3786ftn8gpp8c0000gn/T/ipykernel_37759/1567032424.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created


In [116]:
average_df = feature_engineer(clean_df)

✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ New featured engineered new shape is (950, 56)


In [121]:
average_df.columns

Index(['ID', 'innings', 'overs', 'ballnumber', 'batter', 'bowler',
       'non_striker', 'extra_type', 'batsman_run', 'extras_run', 'total_run',
       'non_boundary', 'isWicketDelivery', 'player_out', 'kind',
       'fielders_involved', 'BattingTeam', 'City', 'Date', 'Season',
       'MatchNumber', 'Team1', 'Team2', 'Venue', 'TossWinner', 'TossDecision',
       'SuperOver', 'WinningTeam', 'WonBy', 'Margin', 'Player_of_Match',
       'Team1Players', 'Team2Players', 'Umpire1', 'Umpire2',
       'Avg_Weighted_Score_Team1', 'Avg_Weighted_Score_Team2',
       'batting_average_PlayersTeam1_weighted',
       'batting_average_PlayersTeam2_weighted',
       'batting_strike_rate_PlayersTeam1_weighted',
       'batting_strike_rate_PlayersTeam2_weighted',
       'bowling_average_PlayersTeam1', 'bowling_average_PlayersTeam2',
       'bowling_economy_rate_PlayersTeam1',
       'bowling_economy_rate_PlayersTeam2', 'bowling_strike_rate_PlayersTeam1',
       'bowling_strike_rate_PlayersTeam2', 'win_ra

In [124]:
team_1_columns = ['Team1', 'Avg_Weighted_Score_Team1', 'batting_average_PlayersTeam1_weighted', 'batting_strike_rate_PlayersTeam1_weighted',\
    'bowling_average_PlayersTeam1', 'bowling_economy_rate_PlayersTeam1', 'bowling_strike_rate_PlayersTeam1', 'win_ratio_PlayersTeam1']

team_2_columns = ['Team2', 'Avg_Weighted_Score_Team2', 'batting_average_PlayersTeam2_weighted', 'batting_strike_rate_PlayersTeam2_weighted',\
    'bowling_average_PlayersTeam2', 'bowling_economy_rate_PlayersTeam2', 'bowling_strike_rate_PlayersTeam2', 'win_ratio_PlayersTeam2']

In [128]:
new_columns = ['team', 'Avg_Weighted_Score', 'batting_average', 'batting_strike_rate', 'bowling_average', 'bowling_economy_rate', \
    'bowling_strike_rate', 'win_ratio']

In [140]:
team1 = df2[team_1_columns].set_axis(new_columns, axis=1)
team2 = df2[team_2_columns].set_axis(new_columns, axis=1)

In [159]:
weighted_df = pd.concat([team1, team2], ignore_index=True)

In [170]:
df['index'] = df.index

In [169]:
weighted_df.sort_index().sort_values(by=['team'])

,team,Avg_Weighted_Score,batting_average,batting_strike_rate,bowling_average,bowling_economy_rate,bowling_strike_rate,win_ratio
884,chennai super kings,20.465631,27.328055,125.343785,22.546364,5.310909,16.244545,0.524545
1485,chennai super kings,22.252220,28.457232,127.971725,19.726364,6.987273,15.256364,0.570909
830,chennai super kings,20.702917,26.469281,122.517289,20.169091,5.575455,15.881818,0.571818
1139,chennai super kings,22.324482,28.724969,130.967894,15.048182,6.627273,11.166364,0.540000
1143,chennai super kings,21.026833,27.574498,130.609152,15.048182,6.627273,11.166364,0.548182
...,...,...,...,...,...,...,...,...
1493,sunrisers hyderabad,22.491873,28.180002,123.023488,27.967273,7.740000,20.174545,0.447273
1123,sunrisers hyderabad,21.786605,26.868910,126.769828,19.022727,6.918182,12.466364,0.440909
330,sunrisers hyderabad,21.058404,27.381187,125.985139,19.184545,7.670909,14.868182,0.484545
574,sunrisers hyderabad,16.447671,21.426494,118.176948,24.077273,6.543636,18.456364,0.476364


In [160]:
weighted_df.groupby('team').first().sort_values(by=index)

NameError: name 'index' is not defined

In [153]:
weighted_file = .to_csv('~/code/patrickevans29/raw_data/weighted_average_data.csv')

AttributeError: module 'pandas' has no attribute 'to_csv'

In [83]:
csv_file = pd.read_csv('~/code/patrickevans29/raw_data/rolling_average_data.csv')

In [84]:
csv_file

,team,average_score,batting_average,batting_strike_rate,bowling_average,bowling_economy,bowling_strike_rate,win_ratio
0,chennai super kings,21.27,21.27,106.92,14.91,4.57,8.73,1.0
1,chennai super kings,19.27,19.27,105.26,12.43,4.52,7.64,1.0
2,chennai super kings,16.91,16.91,81.69,10.63,4.11,6.79,1.0
3,chennai super kings,16.64,16.64,77.18,9.83,4.03,6.34,1.0
4,chennai super kings,16.25,16.25,78.83,9.21,4.05,5.84,0.8
...,...,...,...,...,...,...,...,...
1525,sunrisers hyderabad,13.79,13.79,75.89,6.58,4.50,4.99,0.5
1526,sunrisers hyderabad,13.60,13.60,74.30,6.16,4.41,4.66,0.5
1527,sunrisers hyderabad,13.35,13.35,72.82,6.73,4.34,5.06,0.5
1528,sunrisers hyderabad,13.65,13.65,74.18,6.46,4.42,4.85,0.5


In [85]:
team_1_stats['batting_average'].values

array([14.3])

In [86]:
team_1 = 'chennai super kings'
team_2 = 'sunrisers hyderabad'

In [109]:
team_1 = team_1
team_2 = team_2

csv_file = pd.read_csv('~/code/patrickevans29/raw_data/rolling_average_data.csv')

team_1_stats = csv_file[csv_file['team']==team_1].tail(1)
team_2_stats = csv_file[csv_file['team']==team_2].tail(1)
            
dict = {'Team1': team_1,
        'Team2': team_2,
        'Avg_Weighted_Score_diff': float(team_1_stats['average_score'].values - team_2_stats['average_score'].values), 
        'batting_average_weighted_diff': float(team_1_stats['batting_average'].values - team_2_stats['batting_average'].values),
        'batting_strike_rate_weighted_diff': float(team_1_stats['batting_strike_rate'].values - team_2_stats['batting_strike_rate'].values), 
        'bowling_average_diff': float(team_1_stats['bowling_average'].values - team_2_stats['bowling_average'].values),
        'bowling_economy_rate_diff': float(team_1_stats['bowling_economy'].values - team_2_stats['bowling_economy'].values),
        'win_ratio_diff': float(team_1_stats['win_ratio'].values - team_2_stats['win_ratio'].values),
        'Venue': venue,
        'City': city,
        'TossWinner': toss_winner,
        'TossDecision': toss_decision
        }

venue = 'eden gardens'
city = 'kolkata'
toss_winner = 1 
toss_decision = 'field'

X_pred = pd.DataFrame(dict, index=[0])

/var/folders/4n/3hw1gckd4zg3786ftn8gpp8c0000gn/T/ipykernel_37759/3453276884.py:11: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  'Avg_Weighted_Score_diff': float(team_1_stats['average_score'].values - team_2_stats['average_score'].values),
/var/folders/4n/3hw1gckd4zg3786ftn8gpp8c0000gn/T/ipykernel_37759/3453276884.py:12: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  'batting_average_weighted_diff': float(team_1_stats['batting_average'].values - team_2_stats['batting_average'].values),
/var/folders/4n/3hw1gckd4zg3786ftn8gpp8c0000gn/T/ipykernel_37759/3453276884.py:13: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is d

In [110]:
X_pred

,Team1,Team2,Avg_Weighted_Score_diff,batting_average_weighted_diff,batting_strike_rate_weighted_diff,bowling_average_diff,bowling_economy_rate_diff,win_ratio_diff,Venue,City,TossWinner,TossDecision
0,chennai super kings,sunrisers hyderabad,0.8,0.8,-5.33,-0.51,-0.35,0.0,eden gardens,kolkata,1,field
